In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_model_dir: Path
    test_model_dir: Path
    model_file_name: str
    alpha : float
    l1_ratio : float
    target_column: str

In [ ]:
from mlproj.constants import *
from mlproj.utils.common import read_yaml, create_directories

In [ ]:
class COnfigurationManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH,
                 schema_filepath: Path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories(Path(self.config.artifacts_root))
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer

        create_directories
        ([
            Path (config.root_dir),
            Path (config.train_model_dir),
            Path (config.test_model_dir)
        ])

        model_trainer_config = ModelTrainerConfig(
            root_dir= Path (config.root_dir),
            train_model_dir= Path (config.train_model_dir),
            test_model_dir= Path (config.test_model_dir),
            model_file_name= config.model_file_name,
            alpha = self.params.alpha,
            l1_ratio = self.params.l1_ratio,
            target_column= self.schema.target_column
        )

        return model_trainer_config

In [ ]:
import os
import pandas as pd
from mlproj import logger
from sklearn.linear_model import ElasticNet
import joblib

In [ ]:
class ModelTrainer:
    def __init__(self, model_trainer_config: ModelTrainerConfig):
        self.model_trainer_config = model_trainer_config

    def train_model(self, X_train: pd.DataFrame, y_train: pd.Series) -> ElasticNet:
        train_data = pd.read_csv(self.config.data_ingestion.train_file_path)
        test_data = pd.read_csv(self.config.data_ingestion.test_file_path)

        train_x = train_data.drop(columns=[self.schema.target_column], axis=1)
        train_y = train_data[self.schema.target_column]
        test_x = test_data.drop(columns=[self.schema.target_column], axis=1)
        test_y = test_data[self.schema.target_column]

        lr = ElasticNet(
            alpha=self.model_trainer_config.alpha,
            l1_ratio=self.model_trainer_config.l1_ratio
            random_state=42
        )
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.model_trainer_config.train_model_dir, self.model_trainer_config.model_file_name))

        

In [ ]:
try:
    config = COnfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(model_trainer_config)
    model_trainer_config.train_model()

except Exception as e:  
    logger.exception(e)